MIMIC_Sepsis
=================

# 1 Preparation

To run this document the following requirements must be satisfied:

- Implement the database mimic in **PostgreSQL** and start it. The instruction can be seen [here](https://github.com/MIT-LCP/mimic-code/tree/main/mimic-iv/buildmimic/postgres). (The name of this environment should be **mimiciv**)
- generate useful abstractions of raw MIMIC-IV data. The instruction be seen [here](https://github.com/MIT-LCP/mimic-code/tree/main/mimic-iv/concepts_postgres) 



To create an anaconda environment and install all the required libraries, uncomment following and run:

In [9]:
#!conda create --name mimiciv_sepsis python=3.11
#!conda activate mimiciv_sepsis
#!pip install -r requirements.txt

Run the following cell to connect to the database.

In [10]:
%load_ext autoreload
%autoreload 2

import psycopg2
from psycopg2 import sql
import csv
import pandas as pd
import numpy as np
import os
import shutil
import csv
from datetime import timedelta
from sklearn.impute import KNNImputer
from sklearn.neighbors import KNeighborsRegressor

# implement the username, password and database name
conn = psycopg2.connect(host='', user='', password='', database='mimiciv')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# 2 Extract selected data from the original database 

We extract the `state space` and `action space` respectively from the mimiciv database. The table `itemid_info/mimic4 itemid.csv` lists all the items required.

***Uncomment the following cell if you first time run the code***

In [3]:
# uncomment the this cell if you first time run the code

# Read the SQL file

try:
    with open('sql/select_patients_cohort.sql', 'r') as file0:
        sql_script_select_patients_cohort = file0.read()
        
    with open('sql/state_from_chartevents.sql', 'r') as file1:
        sql_script_state = file1.read()

    with open('sql/action_from_inputevents.sql', 'r') as file2:
        sql_script_action_from_inputevents = file2.read()

    with open('sql/action_from_vasopressors_equivalent_dose.sql', 'r') as file3:
        sql_script_action_from_vasopressors_equivalent_dose = file3.read()

    # Execute the SQL script and create the tables in schema mimiciv_derived_sepsis
    cursor = conn.cursor()
    
    cursor.execute(sql.SQL(sql_script_select_patients_cohort))
    print("mimiciv_derived_sepsis.sepsis_patients_cohort is created")

    cursor.execute(sql.SQL(sql_script_state))
    print("mimiciv_derived_sepsis.sepsis_state is created")

    cursor.execute(sql.SQL(sql_script_action_from_inputevents))
    print("mimiciv_derived_sepsis.sepsis_action_inputevents is created")

    cursor.execute(sql.SQL(sql_script_action_from_vasopressors_equivalent_dose))
    print("mimiciv_derived_sepsis.sepsis_action_vasopressors_equivalent_dose is created")

    conn.commit()
    cursor.close()
    
except (Exception, psycopg2.DatabaseError) as error:
    print("Error executing SQL statement:", error)

mimiciv_derived_sepsis.sepsis_patients_cohort is created
mimiciv_derived_sepsis.sepsis_state is created
mimiciv_derived_sepsis.sepsis_action_inputevents is created
mimiciv_derived_sepsis.sepsis_action_vasopressors_equivalent_dose is created


Get the number of stay_ids

In [11]:
with conn.cursor() as cursor:
    command = "SELECT distinct stay_id FROM mimiciv_derived_sepsis.sepsis_patients_cohort;"
    cursor.execute(command)   
    result = cursor.fetchall()
    stay_ids= [row[0] for row in result]
    num_stay_ids = len(stay_ids)
    print('Number of stay_ids: ' + str(num_stay_ids))
    cursor.close()

Number of stay_ids: 7404


# 3 Data transfer

## 3.1 Data transfer of State Space
We transfer the data of State Space from Postgresql to csv.

In [5]:
with conn.cursor() as cursor:
    command = "SELECT icu.stay_id, a.age, pat.gender, pat.dod FROM mimiciv_derived.age as a  INNER JOIN mimiciv_hosp.patients pat  ON a.subject_id = pat.subject_id INNER JOIN mimiciv_icu.icustays icu ON icu.subject_id = a.subject_id and icu.hadm_id=a.hadm_id INNER JOIN mimiciv_derived_sepsis.sepsis_patients_cohort sepsis ON sepsis.stay_id=icu.stay_id;"
    cursor.execute(command)   
    result = cursor.fetchall()
    cursor.close()
df=pd.DataFrame(result)
df.columns = ['stay_id', 'age', 'gender', 'dod']

#binary
df['dod_b']=df['dod'].apply(lambda x: 1 if x is not None else 0)

# normailzation
df['gender_n']=df['gender'].apply(lambda x: 1 if x=='M' else 0) 
df['age']=df['age'].astype(float)
df['age_n']=(df['age']-df['age'].mean())/(df['age'].std())

minimum = min(df['age_n'])
maximum = max(df['age_n'])
df['age_n'] = (df['age_n'] - minimum)/(maximum-minimum)


df.to_csv('./output/data/data_raw/PatientAttribute.csv',index=0)

In [6]:
# output to /output/data/data_raw/state/{state_name}.csv
from python.data_preprocessing.data_transfer import data_transfer_state

itemid_list_state, label_state = data_transfer_state(conn, num_stay_ids, percent=0.2)

the total number of value is: 48
output:Heartrate.csv                           	percent of stay_id:1.0
output:ABPs.csv                                	percent of stay_id:0.297947055645597
output:NBPs.csv                                	percent of stay_id:0.9777147487844409
output:ABPd.csv                                	percent of stay_id:0.29808211777417615
output:NBPd.csv                                	percent of stay_id:0.9775796866558617
output:ABPm.csv                                	percent of stay_id:0.3024041058887088
output:NBPm.csv                                	percent of stay_id:0.9775796866558617
output:RespiratoryRate.csv                     	percent of stay_id:0.9998649378714208
output:TemperatureF.csv                        	percent of stay_id:0.9706915180983252
output:TemperatureC.csv                        	percent of stay_id:0.09805510534846029
output:PH_A.csv                                	percent of stay_id:0.531334413830362
output:PH_V.csv                     

## 3.2 Data transfer of Action Space

### 3.2.1 Data transfer of Action Space for *IV fluid bolus*

 - IV fluid bolus
   - NaCl_0.9%
   - Dextrose_5%

In [7]:
# output to /output/data/data_raw/action/IV_fluid_bolus/{IV_fluid_bolus_name}.csv
from python.data_preprocessing.data_transfer import data_transfer_action_IV_fluid_bolus

data_transfer_action_IV_fluid_bolus(conn)

output action (IV_fluid_bolus):	NaCl_0_9%.csv
output action (IV_fluid_bolus):	Dextrose_5%.csv


### 3.2.2 Data transfer of Action Space for *Vasopressors*

we directly obtain `vasopressors_equivalent_dose` 

from `mimiciv_derived.norepinephrine_equivalent_dose` 

based on *"Vasopressor dose equivalence: A scoping review and suggested formula" by Goradia et al. 2020*.

In [8]:
# output to /output/data/data_raw/action/vasopressors/vasopressors_equivalent_dose.csv
from python.data_preprocessing.data_transfer import data_transfer_action_vasopressors_equivalent_dose

data_transfer_action_vasopressors_equivalent_dose(conn)

output action (vasopressors): vasopressors_equivalent_dose.csv


# 4 Data Summerization

## 4.1 Hourly Sample on State Space

In [8]:
# output to /output/data/data_hourly_sample/state/stay_id_{selected_id}.csv
from python.data_preprocessing.hourly_sample import hourly_sample_state
import random
import pandas as pd
from tqdm import tqdm
from tqdm.contrib.concurrent import process_map
from multiprocessing import Pool

if os.path.exists('./output/data/data_hourly_sample/state'):shutil.rmtree('./output/data/data_hourly_sample/state')
os.makedirs('./output/data/data_hourly_sample/state', exist_ok=True)

df_state_space = pd.DataFrame()

# FIXME: code for test
# selected_ids = random.sample(stay_ids, 100)
selected_ids = stay_ids
# print(f'Selected stay_id: {selected_ids}')
# selected_id = 31872514

# for selected_id in tqdm(selected_ids):
#     df_state_space=hourly_sample_state(df_state_space,selected_id, itemid_list_state, label_state, k = 10)
    
# df_state_space.reset_index().to_csv(f'./output/data/data_hourly_sample/state/df_state_space.csv')

def process_id(selected_id):
    return hourly_sample_state(df_state_space, selected_id, itemid_list_state, label_state, k=10)

results = process_map(process_id, selected_ids, max_workers=8, chunksize=10) # Set max_workers to the number of cores you want to use

df_state_space = pd.concat(results)
df_state_space.reset_index().to_csv(f'./output/data/data_hourly_sample/state/df_state_space.csv')


  0%|          | 0/7404 [00:00<?, ?it/s]


NameError: name 'itemid_list_state' is not defined

## 4.1.2 combination
combine the data with age, gender, survival situation in patient attribute

In [ ]:
import pandas as pd

df_patient_attribute = pd.read_csv('./output/data/data_raw/PatientAttribute.csv', header=0)
df_patient_attribute = df_patient_attribute.drop(0)

# remove columns 'dod', 'gender', 'age'
df_patient_attribute = df_patient_attribute.drop(['dod', 'gender', 'age'], axis=1)


df_state_space_f = pd.merge(df_patient_attribute, df_state_space, on='stay_id', how='right')
df_state_space_f.to_csv('./output/data/data_hourly_sample/state/df_state_space_f.csv', index=0)


## 4.1.3 state cluster 

In [ ]:
from sklearn.decomposition import NMF
from sklearn.cluster import KMeans

cluster_num = 1250

features = [label_state[itemid] for itemid in itemid_list_state] +['gender_n','age_n']
features.remove('TemperatureC')

# Convert your DataFrame to a matrix
matrix = df_state_space_f[features].fillna(0).values

# Apply NMF
model = NMF(n_components=cluster_num, init='random', random_state=0)
W = model.fit_transform(matrix)
H = model.components_
# print(H.shape)
kmeans_train = KMeans(n_clusters=cluster_num, random_state=0, verbose=True, n_init=5).fit(W)

### 4.1.4 assemble all the data related to state

In [ ]:
state_clusters = kmeans_train.labels_
state_set_final = pd.DataFrame()
state_set_final['stay_id']=df_state_space_f['stay_id']
state_set_final['chartdatetime']=df_state_space_f['chartdatetime']
state_set_final['state']=state_clusters
state_set_final['mortality']=df_state_space_f['dod_b']

#FIXME: still missing: reward ,action

state_set_final.to_csv('./output/data/data_hourly_sample/state/df_state_space_final.csv', index=0)

## 4.2 Hourly Sample on Action Space

### 4.2.1 Hourly sample IV_fluid_bolus for both continuous and discrete action space

In [12]:
from multiprocessing import Pool
from tqdm.contrib.concurrent import process_map
# output to /output/data/data_hourly_sample/action/IV_fluid_bolus/stay_id_{selected_id}.csv
from python.data_preprocessing.hourly_sample import hourly_sample_action_IV_fluid_bolus
if os.path.exists('./output/data/data_hourly_sample/action/IV_fluid_bolus/'):shutil.rmtree('./output/data/data_hourly_sample/action/IV_fluid_bolus/')


# selected_id = 31872514 # more than 72 hours ICU stay 
# print(f'Selected stay_id: {selected_id}')
# hourly_sample_action_IV_fluid_bolus(selected_id)

# count = 0
# for selected_id in stay_ids:
#     try:
#         hourly_sample_action_IV_fluid_bolus(selected_id)
#     except:
#         # print(f'Error with {selected_id}')
#         count += 1
# print(f'Error count: {count}') # 911 out of 7404 stay_ids (12.3%) did not have IV_fluid_bolus. 7404 - 911 = 6493 (87.7%) stay_ids have IV_fluid_bolus

def process_id(selected_id):
    try:
        return hourly_sample_action_IV_fluid_bolus(selected_id)
    except Exception as e:
        return None
    
results = process_map(process_id, stay_ids, max_workers=8, chunksize=10) # Set max_workers to the number of cores you want to use

results = [r for r in results if r is not None]

df_action_space_IV_fluid_bolus = pd.concat(results)
df_action_space_IV_fluid_bolus.to_csv(f'./output/data/data_hourly_sample/action/df_action_space_IV_fluid_bolus.csv', index=False)

count = sum(1 for result in results if result is None)
print(f'Error count: {count}') # TODO 911 out of 7404 stay_ids (12.3%) did not have IV_fluid_bolus. 7404 - 911 = 6493 (87.7%) stay_ids have IV_fluid_bolus

100%|██████████| 7404/7404 [04:47<00:00, 25.73it/s]


Error count: 0


### 4.2.2 Hourly sample vasopressors_equivalent_dose for both continuous and discrete action space

In [13]:
from multiprocessing import Pool
from tqdm.contrib.concurrent import process_map
# output to /output/data/data_hourly_sample/action/vasopressors_equivalent_dose/stay_id_{selected_id}.csv
from python.data_preprocessing.hourly_sample import hourly_sample_action_vasopressors_equivalent_dose
if os.path.exists('./output/data/data_hourly_sample/action/vasopressors_equivalent_dose'):shutil.rmtree('./output/data/data_hourly_sample/action/vasopressors_equivalent_dose')


# selected_id = 31872514 # more than 72 hours ICU stay 
# print(f'Selected stay_id: {selected_id}')
# hourly_sample_action_vasopressors_equivalent_dose(selected_id)

# count = 0
# for selected_id in stay_ids:
#     try:
#         hourly_sample_action_vasopressors_equivalent_dose(selected_id)
#     except:
#         # print(f'Error with {selected_id}')
#         count += 1
# print(f'Error count: {count}') # 4452 out of 7404 stay_ids (60.1%) did not have vasopressors. 7404 - 4452 = 2952 (39.9%) stay_ids have vasopressors

def process_id(selected_id):
    try:
        return hourly_sample_action_vasopressors_equivalent_dose(selected_id)
    except Exception as e:
        return None
    
results = process_map(process_id, stay_ids, max_workers=8, chunksize=10) # Set max_workers to the number of cores you want to use

results = [r for r in results if r is not None]

df_action_space_vasopressors_equivalent_dose = pd.concat(results)
df_action_space_vasopressors_equivalent_dose.to_csv(f'./output/data/data_hourly_sample/action/df_action_space_vasopressors_equivalent_dose.csv', index=False)

count = sum(1 for result in results if result is None)
print(f'Error count: {count}') # TODO 4452 out of 7404 stay_ids (60.1%) did not have vasopressors. 7404 - 4452 = 2952 (39.9%) stay_ids have vasopressors

100%|██████████| 7404/7404 [01:20<00:00, 92.47it/s] 


Error count: 0
